In [10]:
from scdv.utilities.mongodb_dataframe_builder import AtomicServiceDataFrameBuilder
from scdv.utilities.mongodb_handler import atomic_services_collection, atomic_df
import pandas as pd

In [13]:
task_id = 'Activity_0a0lhos'
doc = atomic_df.find_one({'task_id': task_id})

In [14]:


if not doc:
    print(f"❌ Nessun documento trovato per task_id: {task_id}")
else:
    print(f"✅ Documento trovato per task_id: {task_id}")

    # Prendi la lista dentro 'data'
    data_list = doc.get('data', [])

    if not data_list:
        print("⚠️ Nessun dato nella chiave 'data'")
    else:
        # Crea DataFrame unico
        df = pd.DataFrame(data_list)

        # Mostra DataFrame
        print("=== Combined DataFrame ===")
        display(df)

✅ Documento trovato per task_id: Activity_0a0lhos
=== Combined DataFrame ===


,task_id,diagram_id,name,atomic_type,method,url,owner,input_1,input_1_type,input_2,input_2_type,input_3,input_3_type,input_4,input_4_type,output_1,output_1_type,output_2,output_2_type
0,Activity_0a0lhos,687e4466b262d54c9335c8c4,prova,collect,GET,/url1,paolo,a,string,b,string,3.0,float,4.58,float,bici,string,False,boolean
